In [1]:
import cv2
import numpy as np
import glob
import matplotlib.pyplot as plt
from dt_apriltags import Detector
import matplotlib.cm as cm
from lane_detection import *
from lane_following import *


In [2]:
cap = cv2.VideoCapture('AUV_Vid.mkv')
img_array = []
i = 0
ret = True
detectedLane = 0
failedToDetectLane = 0
while ret:
    if i > 1300:
        break
    slopeSet = set() 
    i += 1
    ret, frame = cap.read()
    if(i%1 == 0 and i > 800):
        lines = detect_lines(frame, 30, 90, 3,10,10)
        #print (lines)
    
        #try: 
        #    frame = draw_lines(frame, lines,(0, 255, 0))
        #    print ("tried to draw lines")
        #except :
       #     pass
        try:
            lanes = detect_lanes(lines)
            #print ("tried to detect lanes")
            pickedLane = pick_lane(lanes)
            frame = draw_Single_lane(frame, pickedLane, (255, 0, 0))
            frame = draw_lines(frame, lines,(0, 255, 0))
            center_intercept, center_slope = get_lane_center(pickedLane)
            xPoint = pickedLane[0][2]
            yPoint = pickedLane[0][3]
            cv2.line(frame, (int(center_intercept), 1080), (int(xPoint), int(yPoint)), (0,0,255), 3)
            rec_dir = recommend_direction(center_intercept, center_slope)
            print (rec_dir)
            cv2.putText(frame, rec_dir, (0, 130), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            detectedLane += 1
        except:
            failedToDetectLane += 1
            pass
        
        
        #recommend_direction(center_intercept, center_slope)
        img_array.append(frame)
        height, width, layers = frame.shape
        size = (width,height)
        #plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        #plt.show()
        accuracy = (detectedLane/(detectedLane+failedToDetectLane))*100
        cv2.putText(frame, str(accuracy), (0, 300), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        print(f"accuracy: {accuracy}%")
        print(f"currently on frame: {i}")

out = cv2.VideoWriter('testVid8.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)

for i in range(len(img_array)):
    out.write(img_array[i])
out.release()#1500 2000

        

KeyboardInterrupt: 